In [1]:
from main import *
import tensorflow as tf

from tqdm import tnrange, tqdm_notebook, tqdm

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


In [2]:
num_classes = 2
image_shape = (160, 576)
epochs = 20
batch_size = 5
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

Tests Passed


In [3]:
sess = tf.Session()

In [4]:
# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')

# OPTIONAL: Augment Images for better results
#  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

# TODO: Build NN using load_vgg, layers, and optimize function
input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
learning_rate = tf.placeholder(tf.float32)
correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes))
logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

# Initialize TF variables
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [5]:
get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

In [6]:
for ep in tnrange(epochs, unit=' epochs'):
    pbar = tqdm_notebook(enumerate(get_batches_fn(batch_size)), unit=' batches')
    for batch_idx, (image, label) in pbar:
        feed_dict = {input_image: image,
                     correct_label: label,
                     keep_prob: 0.50,
                     learning_rate: 1e-5,
                     }
        _, loss = sess.run([train_op, cross_entropy_loss], feed_dict)
        if batch_idx % 10 == 0:
            pbar.set_description('Epoch: %d, Batch: %d, Loss = %.3f' % (ep+1, batch_idx, loss))

In [8]:
helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

Training Finished. Saving test images to: ./runs/1504584542.56346


In [7]:
import time
saver = tf.train.Saver()

save_path = saver.save(sess, "./data/tmp/model_%d.ckpt"%(time.time()))
print("Model saved in file: %s" % save_path)

Model saved in file: ./data/tmp/model_1504584513.ckpt


In [10]:
sess.close()